## merge in advance the different ukbb datasets
* filter them to have only cases with any UKB diagnosis (440K out of 500K) - or  keep only those with post 2011 diag = 370K!
    * According to df diag 
* Drop some unwanted columns (smartwatch sleep strings) and munge some others
* drop 1.0 correlated cols
* Save as single parquet file

In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import OrdinalEncoder
from arfs.preprocessing import OrdinalEncoderPandas
from feature_engine.selection import SmartCorrelatedSelection, DropCorrelatedFeatures, DropDuplicateFeatures

# import dcor

In [2]:
def get_coltypes_list(X:pd.DataFrame):
    text_cols_list = X.select_dtypes(["O","string","category"]).columns.tolist()
    m = X.filter(text_cols_list,axis=1).nunique()> 50#20
    list_features_high_cardinality = list(X.filter(text_cols_list,axis=1).nunique()[m].index)

    categorical_cols = X[text_cols_list].columns.difference(list_features_high_cardinality).to_list()

    # categorical_cols = categorical_cols  # Update with your actual categorical columns
    text_cols =  list_features_high_cardinality # Update with your actual text columns
    numerical_cols = X.select_dtypes(include=['number',"boolean"]).columns.difference(categorical_cols + text_cols).to_list()
    print("text_cols",text_cols)

    return categorical_cols,text_cols,numerical_cols
# categorical_cols,text_cols,numerical_cols = get_coltypes_list(X)


In [3]:
DIAG_TIDY_PATH = "ukbb-hack/df_diag_tidy.parquet"
LIPID_HACKATHON_CHOL_BLOOD_RAW_FILE_PATH = "ukbb-hack/lipid_cholesterol_blood_and_medication.csv.gz"

In [4]:
list_cols_replace_isna_feat = ["Fitness test results, including ECG data",
                               "Acceptability of each blow result (text) | Array 0"]
list_sequential_features = ['Light - Day average',
       'Sedentary - Day average', 'Sleep - Day average',
       'Sleep - Weekday hour average', 'Sleep - Weekend hour average',
 'Pulse wave pressure versus time response curve'] # some features that are sequences of numbers - need more processing to use

actually_numeric_cat_cols_list = ['Coffee intake', 'Tea intake', 'Water intake',
'Age when diabetes-related eye disease diagnosed', 'Age diabetes diagnosed','Age heart attack diagnosed']

DROP_FEAT_COLS_LIST = ["cutoff_date","prediction_cutoff_year"] # prediction_cutoff_year - important, maybe leaky feat?

In [5]:
df_diag = pd.read_parquet(DIAG_TIDY_PATH,columns=['eid', 'Diagnosis Date']).drop_duplicates()
print(df_diag.shape[0])
print(df_diag["eid"].nunique())
df_diag.columns

2584043
440004


Index(['eid', 'Diagnosis Date'], dtype='object')

In [6]:
df_diag.loc[df_diag['Diagnosis Date'].dt.year>=2009]["eid"].nunique()

387979

In [7]:
df_diag.loc[df_diag['Diagnosis Date'].dt.year>=2000]["eid"].nunique()

431824

In [8]:
df = df_diag[["eid"]].drop_duplicates().reset_index(drop=True)
df

,eid
0,1000015
1,1000027
2,1000039
3,1000040
4,1000053
...,...
439999,6026238
440000,6026246
440001,6026252
440002,6026289


In [9]:
df_chol =  pd.read_csv(LIPID_HACKATHON_CHOL_BLOOD_RAW_FILE_PATH).dropna(axis=1,thresh=150)
df_chol = df_chol.loc[df_chol["eid"].isin(df_diag["eid"])].set_index("eid")
df_chol = df_chol.filter(like="_i0",axis=1) # keep instance 0 features
df_chol.columns = df_chol.columns.str.replace(r"| Instance 0","",regex=False).str.replace(r"  _i0","",regex=False)
# df_chol

In [10]:
df_chol2 = pd.read_csv("ukbb-hack/Cholesterol_miki_participant.csv.gz").rename(columns={"Participant ID":"eid"},errors="ignore")
# df_chol2["eid"] = df_chol2["Participant ID"]
df_chol2 = df_chol2.loc[df_chol2["eid"].isin(df_diag["eid"])].set_index("eid")
df_chol2 = df_chol2.filter(like="Instance 0",axis=1).dropna(axis=1,thresh=150)
df_chol2.columns = df_chol2.columns.str.replace(r"| Instance 0","",regex=False).str.replace(r"  _i0","",regex=False)
# df_chol2

In [11]:
df_aux =  pd.read_csv("ukbb-hack/Data/data_aux.csv.gz").rename(columns={"Participant ID":"eid"}).dropna(axis=1,thresh=100)
df_aux = df_aux.loc[df_aux["eid"].isin(df_diag["eid"])].set_index("eid")
df_aux.drop(list(df_aux.filter(regex = 'Instance [1-4]')), axis = 1, inplace = True)
df_aux.columns = df_aux.columns.str.replace(r" | Instance 0","",regex=False).str.replace(r"  _i0","",regex=False)
# print(df.merge(df_aux,how="inner",on="eid").shape,"inner join #")
df_aux

,Standard PRS for age at menopause (AAM),Standard PRS for age-related macular degeneration (AMD),Standard PRS for alzheimer's disease (AD),Standard PRS for asthma (AST),Standard PRS for atrial fibrillation (AF),Standard PRS for bipolar disorder (BD),Standard PRS for body mass index (BMI),Standard PRS for bowel cancer (CRC),Standard PRS for breast cancer (BC),Standard PRS for cardiovascular disease (CVD),...,Standard PRS for schizophrenia (SCZ),Standard PRS for systemic lupus erythematosus (SLE),Standard PRS for type 1 diabetes (T1D),Standard PRS for type 2 diabetes (T2D),Standard PRS for ulcerative colitis (UC),Standard PRS for venous thromboembolic disease (VTE),PRS genetic principal components | Array 0,Townsend deprivation index at recruitment,Birth weight,Cascot confidence score
eid,,,,,,,,,,,,,,,,,,,,,
1000015,1.594120,-0.472454,0.318543,-0.608597,0.732505,-1.236850,-1.017890,0.441509,-0.632416,1.234230,...,-0.043712,-0.004664,2.507540,-0.878088,1.101050,2.334520,-8.86839,-4.22,3.20,NaN
1000053,-2.042420,0.823560,-1.271560,-1.745390,-1.313440,-0.586529,1.892600,1.446960,-0.248250,-1.081420,...,-1.664590,-0.751579,-0.401147,-0.445942,0.202860,-1.475670,-9.00802,-3.67,3.63,NaN
1000132,0.047436,0.378806,0.883113,-0.413509,-0.067494,-0.699923,1.727930,-0.728904,-1.332350,0.458169,...,-0.397792,0.519007,1.504330,1.762930,0.080039,0.121405,-9.99725,-5.88,2.78,NaN
1000148,-0.877326,1.132440,-0.493331,-0.255172,0.638398,-0.124744,0.448447,1.559180,-0.217675,-0.584126,...,1.016900,-0.019071,0.300340,-0.660356,-1.143810,0.873744,-8.48041,0.25,3.06,NaN
1000163,-0.765377,0.729507,0.352771,0.024505,0.558936,-0.329746,0.993897,-1.218300,-0.303386,0.906272,...,-0.483350,-0.294396,1.004550,-0.229111,-0.211714,0.067843,-8.50595,-4.94,3.18,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025855,-0.096209,0.060246,1.219290,0.239403,1.377700,-1.924120,0.327673,1.990500,-1.105360,-0.230198,...,-0.759812,-1.308030,-1.001470,-0.332576,3.044820,0.167326,-8.47597,0.20,NaN,NaN
6025869,0.584184,0.699316,-1.036210,0.087223,-0.401291,-1.579890,-0.312191,0.833685,1.492780,0.605246,...,0.114199,-0.153901,1.035620,0.758688,-0.972170,0.743334,-9.81879,0.42,3.91,NaN
6025912,-0.287166,2.429460,-0.579973,1.234190,-0.255566,0.840383,0.196219,2.917120,0.363257,-0.014443,...,1.164330,0.458337,-0.785070,0.388690,1.713490,0.859654,-8.74261,1.94,NaN,NaN


In [12]:
%%time
# moved to top 
## another bunch of ukbb features - might overlap with first? 
df_aux2 =  pd.read_csv("ukbb-hack/Data/data_aux_2.csv.gz",low_memory=False).rename(columns={"Participant ID":"eid"}).dropna(thresh=50,axis=1)
df_aux2 = df_aux2.loc[df_aux2["eid"].isin(df_diag["eid"])].set_index("eid")
df_aux2.columns = df_aux2.columns.str.replace(r" | Instance 0","",regex=False)
df_aux2.drop(list(df_aux2.filter(regex = 'Instance [1-5]')), axis = 1, inplace = True)
df_aux2.columns = df_aux2.columns.str.replace(r" | Instance 0","",regex=False).str.replace(r"  _i0","",regex=False)
for c in list_cols_replace_isna_feat:
    df_aux2[c] = df_aux2[c].isna()
## potentially  Leaky time based feature, since not filtered by instance
df_aux2.drop(columns=["Records of COVID-19 test results"],errors="ignore",inplace=True)

df_aux2.drop(list_sequential_features,axis=1,inplace=True,errors="ignore")
df_aux2.drop(list(df_aux2.filter(regex = 'Instance [1-5]')), axis = 1, inplace = True) # Future instances are leaks currently - drop them

print(df_aux2.shape)
# print(df.merge(df_aux2,how="inner",on="eid").shape,"inner join #")
# df = df.merge(df_aux2,how="left",on="eid", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').dropna(thresh=50,axis=1)
df_aux2

(439796, 189)
CPU times: user 15.4 s, sys: 1.53 s, total: 16.9 s
Wall time: 16.9 s


,Overall health rating,"Long-standing illness, disability or infirmity",Falls in the last year,Weight change compared with 1 year ago,General pain for 3+ months,Fluid intelligence score,"Medication for cholesterol, blood pressure, diabetes, or take exogenous hormones",Eye problems/disorders,Non-accidental death in close genetic family,"Medication for cholesterol, blood pressure or diabetes",...,"Non-cancer illness code, self-reported | Array 5","Non-cancer illness code, self-reported | Array 6",Date of all cause dementia report,Date of alzheimer's disease report,Date of vascular dementia report,Behavioural and miscellaneous addictions,Ever addicted to any substance or behaviour,Ongoing addiction or dependence on illicit or recreational drugs,Ever addicted to a behaviour or miscellanous,Ever addicted to illicit or recreational drugs
eid,,,,,,,,,,,,,,,,,,,,,
1000015,Good,Do not know,Only one fall,Yes - lost weight,NaN,NaN,None of the above,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000053,Good,No,No falls,Yes - gained weight,NaN,NaN,None of the above,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000132,Good,Yes,No falls,Yes - lost weight,NaN,10.0,NaN,None of the above,No,None of the above,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
1000148,Fair,No,More than one fall,Yes - gained weight,NaN,NaN,Blood pressure medication,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000163,Excellent,No,No falls,No - weigh about the same,NaN,NaN,NaN,NaN,NaN,None of the above,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025855,Excellent,No,No falls,No - weigh about the same,NaN,NaN,None of the above,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6025869,Fair,No,No falls,No - weigh about the same,NaN,7.0,NaN,None of the above,No,Cholesterol lowering medication|Blood pressure...,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
6025912,Fair,No,No falls,No - weigh about the same,NaN,NaN,NaN,NaN,NaN,None of the above,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.dropna(axis=1,thresh=30,inplace=True)
print(df.shape)
df = df.merge(df_chol,how="left",on="eid", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
df = df.merge(df_chol2,how="left",on="eid", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
print(df.shape," before aux")
df = df.merge(df_aux,how="left",on="eid", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
print(df.shape," After aux")
df = df.merge(df_aux2,how="left",on="eid", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
print(df.shape[1]," After aux2")
df.drop(list_sequential_features,axis=1,inplace=True,errors="ignore")
df.drop(list(df.filter(regex = 'Instance [1-4]')), axis = 1, inplace = True) # Future instances are leaks currently - drop them
del df_aux,df_chol,df_chol2,df_aux2

df = df.dropna(thresh=50,axis=1).drop_duplicates()
print(df.shape)
df

(440004, 1)
(440004, 43)  before aux
(440004, 83)  After aux
270  After aux2
(440004, 270)


,eid,Apolipoprotein A Blood biochemistry,Apolipoprotein B Blood biochemistry,Cholesterol Blood biochemistry,HDL cholesterol Blood biochemistry,LDL direct Blood biochemistry,Lipoprotein A Blood biochemistry,Triglycerides Blood biochemistry,"Medication for cholesterol, blood pressure or diabetes Medication",Average Diameter for HDL Particles,...,"Non-cancer illness code, self-reported | Array 5","Non-cancer illness code, self-reported | Array 6",Date of all cause dementia report,Date of alzheimer's disease report,Date of vascular dementia report,Behavioural and miscellaneous addictions,Ever addicted to any substance or behaviour,Ongoing addiction or dependence on illicit or recreational drugs,Ever addicted to a behaviour or miscellanous,Ever addicted to illicit or recreational drugs
0,1000015,NaN,1.066,6.173,NaN,3.741,4.60,1.831,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,1.126,0.697,3.506,0.966,2.217,NaN,1.081,Cholesterol lowering medication,9.5004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,1.394,0.685,4.296,1.352,2.321,14.98,1.005,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
3,1000040,1.779,0.835,5.036,1.495,2.991,NaN,2.587,Cholesterol lowering medication|Blood pressure...,9.5772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,2.134,0.934,6.157,2.152,3.348,29.50,1.876,NaN,10.1220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439999,6026238,1.038,0.602,3.176,0.805,1.922,5.08,1.400,None of the above,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440000,6026246,1.342,1.432,7.152,1.220,5.039,25.90,1.909,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
440001,6026252,NaN,1.031,4.738,NaN,3.208,7.30,2.115,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440002,6026289,1.372,0.995,4.878,1.180,3.214,NaN,0.902,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# df["Diagnosis date"]

In [15]:
for c in actually_numeric_cat_cols_list:
    if c in df.select_dtypes(["O","string","category"]).columns:
        
        # X[c] = pd.to_numeric(X[c].str.replace("Less than one","0").str.replace("Do not know","-1").str.replace("Unknown","-1"),errors="coerce")
        df[c] = pd.to_numeric(df[c].str.replace("Less than one","0").str.replace("Do not know","-1").str.replace("Unknown","-1"),errors="coerce")

## drop some sequential cols
df.drop(list_sequential_features,axis=1,inplace=True,errors="ignore")

In [16]:
categorical_cols,list_features_high_cardinality,numerical_cols = get_coltypes_list(df)

text_cols ['Eye problems/disorders', 'Country of Birth (non-UK origin)', 'Mineral and other dietary supplements', 'Job code at visit', 'Illnesses of mother', 'Illnesses of siblings', 'Illnesses of adopted father', 'Date of attending assessment centre', 'Qualifications', 'Year immigrated to UK (United Kingdom)', 'Mouth/teeth dental problems', 'Most recent bowel cancer screening', 'Treatment/medication code | Array 0', 'Age cataract diagnosed', 'Age macular degeneration diagnosed', 'Age other serious eye condition diagnosed', 'Age angina diagnosed', 'Age high blood pressure diagnosed', 'Age deep-vein thrombosis (DVT, blood clot in leg) diagnosed', 'Age emphysema/chronic bronchitis diagnosed', 'Age asthma diagnosed', 'Mental health conditions ever diagnosed by a professional', 'Age hay fever, rhinitis or eczema diagnosed', 'Non-cancer illness code, self-reported | Array 0', 'Non-cancer illness code, self-reported | Array 1', 'Non-cancer illness code, self-reported | Array 2', 'Non-cancer 

In [17]:
# %%time
# ## ~ 10 min w spearman, , 1 hour with method=dcor.distance_correlation
# ## almost all features kept... pointless? (with thresh - threshold= 0.9999,method= "spearman",)
# cfs = SmartCorrelatedSelection(threshold= 0.9999,
#                                # threshold= 1.0
#                                method=dcor.distance_correlation ## slower but more flexible, and leaves more cases
#                                # method= "spearman", #"kendall",
#                                # method= "kendall",
#                               ).fit(OrdinalEncoderPandas().fit_transform(df.drop(columns=list_features_high_cardinality,errors="ignore")))
#     # keep_feats = list(set(cfs.get_feature_names_out()+["Sex","Age"]+X.select_dtypes(["O","string","category","boolean"]).columns.tolist()))
# keep_feats = ["eid"] + list(set(cfs.get_feature_names_out()+["Sex","age"]+list_features_high_cardinality)) ##  ["Sex","age"] not in this data

# df = df.filter(keep_feats,axis=1)
# df

In [18]:
df

,eid,Apolipoprotein A Blood biochemistry,Apolipoprotein B Blood biochemistry,Cholesterol Blood biochemistry,HDL cholesterol Blood biochemistry,LDL direct Blood biochemistry,Lipoprotein A Blood biochemistry,Triglycerides Blood biochemistry,"Medication for cholesterol, blood pressure or diabetes Medication",Average Diameter for HDL Particles,...,"Non-cancer illness code, self-reported | Array 5","Non-cancer illness code, self-reported | Array 6",Date of all cause dementia report,Date of alzheimer's disease report,Date of vascular dementia report,Behavioural and miscellaneous addictions,Ever addicted to any substance or behaviour,Ongoing addiction or dependence on illicit or recreational drugs,Ever addicted to a behaviour or miscellanous,Ever addicted to illicit or recreational drugs
0,1000015,NaN,1.066,6.173,NaN,3.741,4.60,1.831,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,1.126,0.697,3.506,0.966,2.217,NaN,1.081,Cholesterol lowering medication,9.5004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,1.394,0.685,4.296,1.352,2.321,14.98,1.005,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
3,1000040,1.779,0.835,5.036,1.495,2.991,NaN,2.587,Cholesterol lowering medication|Blood pressure...,9.5772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,2.134,0.934,6.157,2.152,3.348,29.50,1.876,NaN,10.1220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439999,6026238,1.038,0.602,3.176,0.805,1.922,5.08,1.400,None of the above,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440000,6026246,1.342,1.432,7.152,1.220,5.039,25.90,1.909,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN
440001,6026252,NaN,1.031,4.738,NaN,3.208,7.30,2.115,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440002,6026289,1.372,0.995,4.878,1.180,3.214,NaN,0.902,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### todo: do this in feature extraction? (Since it's future info/may be needed for features
* do keep the `Date is unknown` fix!

In [19]:
for c in df.columns:
    if "date" in c.lower():
        try:
            df[c] = df[c].str.replace("Date is unknown","1899-01-01") # handle missing dates - stil filterable out, but put in past	
            df[c] = pd.to_datetime(df[c])
            print(c)
        except:print("Failed",c)
#     ## Skip for now - leaky features!  
        # df[f"{c.lower().replace('date','date exists')}"] = ~df[c].isna()
        # print(df[f'{c.lower().replace("date","date exists")}'].agg(["sum","mean"]).round(3))
## alt: 
### get if date is after instance 0 / first Date of attending assessment centre	
date_cols_list = df.select_dtypes(["datetime"]).columns.difference(["Date of attending assessment centre","instance0_date","cutoff_date"]).tolist()
for c in date_cols_list:
    df[f"{c.replace('Date','Date elapsed')}"] = np.where(df[c]<=df["Date of attending assessment centre"],
                                                         df["Date of attending assessment centre"].sub(df[c]).dt.days//365,np.NaN) #-1) # months before, if exists, otherwise -1

Date of attending assessment centre
Date of all cause dementia report
Date of alzheimer's disease report
Date of vascular dementia report


In [20]:
df.select_dtypes(["datetime"])

,Date of attending assessment centre,Date of all cause dementia report,Date of alzheimer's disease report,Date of vascular dementia report
0,2008-05-02,NaT,NaT,NaT
1,2008-01-29,NaT,NaT,NaT
2,2009-05-06,NaT,NaT,NaT
3,2009-12-10,NaT,NaT,NaT
4,2007-12-17,NaT,NaT,NaT
...,...,...,...,...
439999,2010-06-15,NaT,NaT,NaT
440000,2009-08-15,NaT,NaT,NaT
440001,2009-08-24,NaT,NaT,NaT
440002,2009-10-12,NaT,NaT,NaT


In [21]:
df.columns[df.columns.str.contains("part")]

Index(['Weight(participant - p21002_i0)', 'Weight(participant - p23098_i0)',
       'Body mass index (BMI)(participant - p21001_i0)',
       'Body mass index (BMI)(participant - p23104_i0)'],
      dtype='object')

## remove "participant and i0 from bmi, weight

In [22]:
df.columns = df.columns.str.replace(r"(participant - ",r" (").str.replace(r"_i0","",regex=False).str.strip()

In [23]:
df.columns[df.columns.str.contains("BM")]

Index(['Standard PRS for body mass index (BMI)',
       'Standard PRS for estimated bone mineral density t-score (EBMDT)',
       'Enhanced PRS for estimated bone mineral density t-score (EBMDT)',
       'Body mass index (BMI) (p21001)', 'Body mass index (BMI) (p23104)'],
      dtype='object')

In [25]:
import pandas as pd
import re

def rearrange_column_name(col_name):
        """
    Swaps potential acronyms to start of text. e.g. "(BMI)"
    Swap potential acronyms/ substrings in parentheses only if they contain at least one capitalized letter.
    For outputs containing "PRS for", replace "PRS for" with an empty string and add "genetics" at the end of the text.
    """
    # Find all substrings in parentheses
    substrings_in_parentheses = re.findall(r'\([^\)]*\)', col_name)
    substring_to_move = None
    # Process substrings in order
    for substring in substrings_in_parentheses:
        content = substring[1:-1]  # Remove parentheses
        # # Check if the content has at least one capital letter or length > 10
        # if re.search(r'[A-Z]', content) or len(content) > 10:
        if re.search(r'[A-Z]', content):
            substring_to_move = substring
            break
    # Remove 'Standard PRS for', 'Enhanced PRS for', or 'PRS for' from the start
    col_name_without_prefix = re.sub(r'^(Standard|Enhanced)\s+PRS for', '', col_name).strip()
    if substring_to_move:
        # Remove the substring from its current position
        col_name_without_substring = col_name_without_prefix.replace(substring_to_move, '').strip()
        # Move the substring to the front
        new_col_name = f'{substring_to_move} {col_name_without_substring}'
    else:
        new_col_name = col_name_without_prefix
    # If 'PRS for' was in the original col_name, add 'genetics' at the end
    if re.search(r'(Standard|Enhanced)\s+PRS for', col_name):
        new_col_name = f'{new_col_name} genetic risk'
    # Strip leading/trailing whitespace and reduce multiple spaces to single spaces
    new_col_name = re.sub(r'\s+', ' ', new_col_name.strip())
    return new_col_name

original = df.columns
# Apply the function to each column name in your DataFrame 'df'
df.columns = [rearrange_column_name(col) for col in df.columns]

df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
# # Display the results
for original, new in zip(original,df.columns.to_list()):
    if original != new:
        print(f"Original: {original}\nModified: {new}\n")

s1 = df.shape[1]
print(s1)
# Remove duplicate columns by keeping the first occurrence
df = df.loc[:, ~df.columns.duplicated()]
print(s1-df.shape[1],"duplicate renamed cols dropped")

Original: Standard PRS for age at menopause (AAM)
Modified: (AAM) age at menopause genetic risk

Original: Standard PRS for age-related macular degeneration (AMD)
Modified: (AMD) age-related macular degeneration genetic risk

Original: Standard PRS for alzheimer's disease (AD)
Modified: (AD) alzheimer's disease genetic risk

Original: Standard PRS for asthma (AST)
Modified: (AST) asthma genetic risk

Original: Standard PRS for atrial fibrillation (AF)
Modified: (AF) atrial fibrillation genetic risk

Original: Standard PRS for bipolar disorder (BD)
Modified: (BD) bipolar disorder genetic risk

Original: Standard PRS for body mass index (BMI)
Modified: (BMI) body mass index genetic risk

Original: Standard PRS for bowel cancer (CRC)
Modified: (CRC) bowel cancer genetic risk

Original: Standard PRS for breast cancer (BC)
Modified: (BC) breast cancer genetic risk

Original: Standard PRS for cardiovascular disease (CVD)
Modified: (CVD) cardiovascular disease genetic risk

Original: Standard

In [26]:
df

,eid,Apolipoprotein A Blood biochemistry,Apolipoprotein B Blood biochemistry,Cholesterol Blood biochemistry,HDL cholesterol Blood biochemistry,LDL direct Blood biochemistry,Lipoprotein A Blood biochemistry,Triglycerides Blood biochemistry,"Medication for cholesterol, blood pressure or diabetes Medication",Average Diameter for HDL Particles,...,Date of alzheimer's disease report,Date of vascular dementia report,Behavioural and miscellaneous addictions,Ever addicted to any substance or behaviour,Ongoing addiction or dependence on illicit or recreational drugs,Ever addicted to a behaviour or miscellanous,Ever addicted to illicit or recreational drugs,Date elapsed of all cause dementia report,Date elapsed of alzheimer's disease report,Date elapsed of vascular dementia report
0,1000015,NaN,1.066,6.173,NaN,3.741,4.60,1.831,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,1.126,0.697,3.506,0.966,2.217,NaN,1.081,Cholesterol lowering medication,9.5004,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,1.394,0.685,4.296,1.352,2.321,14.98,1.005,NaN,NaN,...,NaT,NaT,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,1.779,0.835,5.036,1.495,2.991,NaN,2.587,Cholesterol lowering medication|Blood pressure...,9.5772,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,2.134,0.934,6.157,2.152,3.348,29.50,1.876,NaN,10.1220,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439999,6026238,1.038,0.602,3.176,0.805,1.922,5.08,1.400,None of the above,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440000,6026246,1.342,1.432,7.152,1.220,5.039,25.90,1.909,NaN,NaN,...,NaT,NaT,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
440001,6026252,NaN,1.031,4.738,NaN,3.208,7.30,2.115,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440002,6026289,1.372,0.995,4.878,1.180,3.214,NaN,0.902,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
pd.to_datetime(df["Date of all cause dementia report"])

0        NaT
1        NaT
2        NaT
3        NaT
4        NaT
          ..
439999   NaT
440000   NaT
440001   NaT
440002   NaT
440003   NaT
Name: Date of all cause dementia report, Length: 440004, dtype: datetime64[ns]

In [29]:
df["eid"].nunique()

440004

In [30]:
# df["Diagnosis Date"] # add here? # actually present only in timestamped diags

In [31]:
%%time
## save - in higher level folder
df.to_parquet("df_ukbb_aux_tidy.parquet") 

CPU times: user 2.45 s, sys: 144 ms, total: 2.59 s
Wall time: 2.53 s


In [32]:
df

,eid,Apolipoprotein A Blood biochemistry,Apolipoprotein B Blood biochemistry,Cholesterol Blood biochemistry,HDL cholesterol Blood biochemistry,LDL direct Blood biochemistry,Lipoprotein A Blood biochemistry,Triglycerides Blood biochemistry,"Medication for cholesterol, blood pressure or diabetes Medication",Average Diameter for HDL Particles,...,Date of alzheimer's disease report,Date of vascular dementia report,Behavioural and miscellaneous addictions,Ever addicted to any substance or behaviour,Ongoing addiction or dependence on illicit or recreational drugs,Ever addicted to a behaviour or miscellanous,Ever addicted to illicit or recreational drugs,Date elapsed of all cause dementia report,Date elapsed of alzheimer's disease report,Date elapsed of vascular dementia report
0,1000015,NaN,1.066,6.173,NaN,3.741,4.60,1.831,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,1.126,0.697,3.506,0.966,2.217,NaN,1.081,Cholesterol lowering medication,9.5004,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,1.394,0.685,4.296,1.352,2.321,14.98,1.005,NaN,NaN,...,NaT,NaT,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,1.779,0.835,5.036,1.495,2.991,NaN,2.587,Cholesterol lowering medication|Blood pressure...,9.5772,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,2.134,0.934,6.157,2.152,3.348,29.50,1.876,NaN,10.1220,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439999,6026238,1.038,0.602,3.176,0.805,1.922,5.08,1.400,None of the above,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440000,6026246,1.342,1.432,7.152,1.220,5.039,25.90,1.909,NaN,NaN,...,NaT,NaT,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
440001,6026252,NaN,1.031,4.738,NaN,3.208,7.30,2.115,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440002,6026289,1.372,0.995,4.878,1.180,3.214,NaN,0.902,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# df = pd.read_parquet("df_ukbb_aux_tidy.parquet")

In [34]:
# df.columns[df.columns.str.contains("smok",case=False)] # 'Amount of tobacco currently smoked'

In [35]:
# df.columns[df.columns.str.contains("pack",case=False)]